# TripAdvisor

### Ejercicios de pandas

1. País con más restaurantes
2. Top 5 mejores restaurantes de cada país
3. Premios (awards) de un restaurant
4. Restaurantes vegetarian_friendly, vegan_options y gluten_free: 
    Cantidad, porcentaje, combinación de varios, gráficos
5. Ubicación de los restaurantes en el mapa de un país dado
6. Determinar las principales características de los restaurantes (features que más se repiten)
7. Restaurantes agrupados por ciudad y ordenados por rango de precio decreciente
8. Porcentaje de restaurantes que sirven desayunos (Breakfast)


### EDA

**Objetivo:** Determinar el mejor restaurant de cada ciudad de un país dado. 

- ¿Qué tienen en común esos restaurantes?
- ¿Qué información es la que más influye en el ranting?

In [1]:
# Librerías que necesitamos
import numpy as np
import pandas as pd

In [4]:
%%time 
# Carga del archivo

# Se hace la primera vez nada más, luego cargamos el archivo parquet que es menos pesado

# Usamos low_memory=False para evitar el warning de algunas columnas que pandas no puede
# determinar el tipo
#df_ta = pd.read_csv('tripadvisor_european_restaurants.csv', encoding='utf8', low_memory=False)

#df_ta

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [5]:
# Guardamos el resultado en un archivo Parquet que podrá ser utilizado luego
#df_ta.to_parquet("tripadvisor.parquet", compression='gzip')

In [2]:
%%time 
df_ta = pd.read_parquet('tripadvisor.parquet', engine='fastparquet')

CPU times: user 8.15 s, sys: 906 ms, total: 9.05 s
Wall time: 10.7 s


In [3]:
df_ta.describe()

,latitude,longitude,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere
count,1.067607e+06,1.067607e+06,593832.000000,593832.000000,593832.000000,986761.000000,1.031162e+06,988204.000000,988204.000000,988204.000000,988204.000000,988204.000000,988204.000000,599325.000000,604287.000000,602692.000000,261785.000000
mean,4.656718e+01,5.838040e+00,6.327081,62.023282,7.630754,4.035943,1.028890e+02,44.563415,24.653440,10.490516,4.109302,2.355306,2.954850,4.104179,4.067245,3.982897,3.933682
std,5.882611e+00,8.639410e+00,0.966734,30.538134,2.550128,0.713694,2.672415e+02,148.728179,89.850801,35.512256,15.669631,9.352756,11.030069,0.564208,0.581267,0.577605,0.555262
min,2.764031e+01,-7.121809e+01,1.000000,0.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,4.190986e+01,-8.027315e-01,6.000000,39.000000,6.000000,3.500000,6.000000e+00,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000,3.500000,3.500000
50%,4.658510e+01,5.646530e+00,7.000000,58.500000,7.000000,4.000000,2.400000e+01,7.000000,3.000000,2.000000,1.000000,0.000000,0.000000,4.000000,4.000000,4.000000,4.000000
75%,5.140537e+01,1.223767e+01,7.000000,81.500000,7.000000,4.500000,9.300000e+01,26.000000,13.000000,6.000000,2.000000,1.000000,2.000000,4.500000,4.500000,4.500000,4.500000
max,6.994156e+01,3.336942e+01,7.000000,168.000000,15.000000,5.000000,5.240400e+04,15229.000000,9383.000000,4091.000000,2132.000000,1253.000000,1215.000000,5.000000,5.000000,5.000000,5.000000


In [4]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083397 entries, 0 to 1083396
Data columns (total 42 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   restaurant_link                    1083397 non-null  object 
 1   restaurant_name                    1083397 non-null  object 
 2   original_location                  1083397 non-null  object 
 3   country                            1083397 non-null  object 
 4   region                             1033074 non-null  object 
 5   province                           742765 non-null   object 
 6   city                               682712 non-null   object 
 7   address                            1083397 non-null  object 
 8   latitude                           1067607 non-null  float64
 9   longitude                          1067607 non-null  float64
 10  claimed                            1081555 non-null  object 
 11  awards                  

### 1. País con más restaurantes

In [17]:
df_ta['country'].value_counts().head(1)

Italy    224763
Name: country, dtype: int64

### 2. Top 5 mejores restaurantes de cada país

In [21]:
df_5_best_country = pd.DataFrame()

for country in df_ta['country'].sort_values().unique():
    df_res = df_ta[df_ta['country'] == country].sort_values('avg_rating', ascending=False)[['country', 'restaurant_name', 'city', 'avg_rating' ]].head(5)   
    df_5_best_country = df_5_best_country.append(df_res)
    
print(df_5_best_country)
    

        country                    restaurant_name           city  avg_rating
300413  Austria                      Curry & Pasta  Koenigstetten         5.0
309306  Austria                 el Feo, Mezcaleria         Vienna         5.0
309343  Austria  Antika - Mazedonisches Restaurant         Vienna         5.0
309342  Austria                       Leckerbissen         Vienna         5.0
309339  Austria                        Groissboeck         Vienna         5.0
...         ...                                ...            ...         ...
604153    Wales                       Cegin Carwyn     Llangeitho         5.0
556565    Wales                        Social Eats        Cardiff         5.0
556551    Wales             Calabrisella Gelateria        Cardiff         5.0
556552    Wales            Canton Chinese Takeaway        Cardiff         5.0
639649    Wales                          AJ's Cafe           Rhyl         5.0

[120 rows x 4 columns]


### 3. Premios (awards) de un restaurant

In [23]:
# Creamos una lista de todos los posibles premios
awards = df_ta.awards.unique().tolist()
aw_list = []

for aw in awards:
    aw_list = aw_list + str(aw).split(', ')
    
# Eliminamos los dobles
s = set(aw_list)
awards_list = list(s)
#Eliminamos nan o None
awards_list.pop(awards_list.index('None'))
awards_list.sort()
awards_list

['Certificate of Excellence 2011',
 'Certificate of Excellence 2012',
 'Certificate of Excellence 2013',
 'Certificate of Excellence 2014',
 'Certificate of Excellence 2015',
 'Certificate of Excellence 2016',
 'Certificate of Excellence 2017',
 'Certificate of Excellence 2018',
 'Certificate of Excellence 2019',
 'Certificate of Excellence 2020',
 'Michelin 2020 for Bib Gourmand: good quality',
 'Michelin 2020 for Comfortable restaurant',
 'Michelin 2020 for Extremely comfortable restaurant.',
 'Michelin 2020 for Luxurious restaurant',
 'Michelin 2020 for One Michelin Star: High quality cooking',
 'Michelin 2020 for Simple restaurant',
 'Michelin 2020 for The Michelin Plate: Good cooking',
 'Michelin 2020 for Three Michelin Stars: Exceptional cuisine',
 'Michelin 2020 for Two Michelin Stars: Excellent cooking',
 'Michelin 2020 for Very comfortable restaurant',
 'Michelin 2021 for Bib Gourmand: good quality',
 'Michelin 2021 for Comfortable restaurant',
 'Michelin 2021 for Extremely co

In [28]:
# Creamos un nuevo DF con los restaurantes y los premios que tiene
# Cada fila será un restaurant y cada columna es un premio

# Creamos un nuevo DF con el nombre del restaurant y sus premios 
# solo con los restaurantes que tienen premios
with_aw = pd.notnull(df_ta["awards"])
df_waw = df_ta[with_aw][['restaurant_name', 'awards']]

# Agregamos una columna por cada premio inicializada a 0
df_waw[awards_list] = 0

df_waw

,restaurant_name,awards,Certificate of Excellence 2011,Certificate of Excellence 2012,Certificate of Excellence 2013,Certificate of Excellence 2014,Certificate of Excellence 2015,Certificate of Excellence 2016,Certificate of Excellence 2017,Certificate of Excellence 2018,...,Michelin 2021 for Three Michelin Stars: Exceptional cuisine,Michelin 2021 for Two Michelin Stars: Excellent cooking,Michelin 2021 for Very comfortable restaurant,Travellers' Choice,Travellers' Choice 2020,Travellers' Choice Best of the Best,good value cooking,worth a detour!,worth a special journey!,worth a stop!
5,L'Auberge Du Vieux Crozet,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,L'entre 2,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Chez Claudine,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,L'Antidote,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,La terrasse,"Certificate of Excellence 2018, Certificate of...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083371,Christopher Pub,Certificate of Excellence 2017,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1083376,Restaurant Radsor,"Certificate of Excellence 2019, Certificate of...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1083378,La NOI,Certificate of Excellence 2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1083384,Sub Cetate Sergiana,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
%%time 
# Calculamos los valores para cada columna 
# Más rápido que línea por línea

# Función para calcular el valor de una columna de premio
# Pasamos la lista de premios del restaurant y el premio a tratar
def aw_by_rest(aw_restau, aw):
    return 1 if aw in aw_restau else 0

for aw in awards_list:
    print(aw)
    df_waw[aw]=df_waw.apply(lambda row: aw_by_rest(row['awards'], aw), axis=1) 
    
# Borramos la columna awards que ya no necesitamos   
df_waw = df_waw.drop(columns=['awards'])
df_waw

Certificate of Excellence 2011
Certificate of Excellence 2012
Certificate of Excellence 2013
Certificate of Excellence 2014
Certificate of Excellence 2015
Certificate of Excellence 2016
Certificate of Excellence 2017
Certificate of Excellence 2018
Certificate of Excellence 2019
Certificate of Excellence 2020
Michelin 2020 for Bib Gourmand: good quality
Michelin 2020 for Comfortable restaurant
Michelin 2020 for Extremely comfortable restaurant.
Michelin 2020 for Luxurious restaurant
Michelin 2020 for One Michelin Star: High quality cooking
Michelin 2020 for Simple restaurant
Michelin 2020 for The Michelin Plate: Good cooking
Michelin 2020 for Three Michelin Stars: Exceptional cuisine
Michelin 2020 for Two Michelin Stars: Excellent cooking
Michelin 2020 for Very comfortable restaurant
Michelin 2021 for Bib Gourmand: good quality
Michelin 2021 for Comfortable restaurant
Michelin 2021 for Extremely comfortable restaurant.
Michelin 2021 for Luxurious restaurant
Michelin 2021 for One Micheli

,restaurant_name,Certificate of Excellence 2011,Certificate of Excellence 2012,Certificate of Excellence 2013,Certificate of Excellence 2014,Certificate of Excellence 2015,Certificate of Excellence 2016,Certificate of Excellence 2017,Certificate of Excellence 2018,Certificate of Excellence 2019,...,Michelin 2021 for Three Michelin Stars: Exceptional cuisine,Michelin 2021 for Two Michelin Stars: Excellent cooking,Michelin 2021 for Very comfortable restaurant,Travellers' Choice,Travellers' Choice 2020,Travellers' Choice Best of the Best,good value cooking,worth a detour!,worth a special journey!,worth a stop!
5,L'Auberge Du Vieux Crozet,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,L'entre 2,0,0,0,0,0,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
9,Chez Claudine,0,0,0,0,1,1,1,1,1,...,0,0,0,1,0,0,0,0,0,0
17,L'Antidote,0,0,0,0,1,1,1,1,1,...,0,0,0,1,0,0,0,0,0,0
18,La terrasse,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083371,Christopher Pub,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1083376,Restaurant Radsor,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1083378,La NOI,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1083384,Sub Cetate Sergiana,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [29]:
# Agregamos los otros restaurantes sin premios       
wo_aw = pd.isnull(df_ta["awards"])
df_woaw = df_ta[wo_aw][['restaurant_name']]

df_woaw[awards_list] = 0
df_woaw

df_aw = pd.concat([df_waw, df_woaw])

print(df_aw.info())

# Cambiamos el tipo de dato a int8 y pasamos de más de 300Mb de memoria a menos de 55Mb
df_aw[awards_list] = df_aw[awards_list].astype(np.int8)

print(df_aw.info())

# Ordenamos el DF
df_aw = df_aw.sort_index()
df_aw.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1083397 entries, 5 to 1083396
Data columns (total 39 columns):
 #   Column                                                       Non-Null Count    Dtype 
---  ------                                                       --------------    ----- 
 0   restaurant_name                                              1083397 non-null  object
 1   awards                                                       263133 non-null   object
 2   Certificate of Excellence 2011                               1083397 non-null  int64 
 3   Certificate of Excellence 2012                               1083397 non-null  int64 
 4   Certificate of Excellence 2013                               1083397 non-null  int64 
 5   Certificate of Excellence 2014                               1083397 non-null  int64 
 6   Certificate of Excellence 2015                               1083397 non-null  int64 
 7   Certificate of Excellence 2016                               10

,restaurant_name,awards,Certificate of Excellence 2011,Certificate of Excellence 2012,Certificate of Excellence 2013,Certificate of Excellence 2014,Certificate of Excellence 2015,Certificate of Excellence 2016,Certificate of Excellence 2017,Certificate of Excellence 2018,...,Michelin 2021 for Three Michelin Stars: Exceptional cuisine,Michelin 2021 for Two Michelin Stars: Excellent cooking,Michelin 2021 for Very comfortable restaurant,Travellers' Choice,Travellers' Choice 2020,Travellers' Choice Best of the Best,good value cooking,worth a detour!,worth a special journey!,worth a stop!
0,Le 147,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Le Saint Jouvent,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Au Bout du Pont,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Le Relais de Naiade,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Relais Du MontSeigne,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,L'Auberge Du Vieux Crozet,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Cafe Restaurant NouLou,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,L'entre 2,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Noste Courtiu,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Chez Claudine,"Travellers' Choice, Certificate of Excellence ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Guardamos el resultado en un archivo Parquet que podrá ser utilizado luego
df_aw.to_parquet("awards.parquet", compression='gzip')


### 4.Restaurantes vegetarian_friendly, vegan_options y gluten_free: Cantidad, porcentaje, combinación de varios, gráficos

### 5. Ubicación de los restaurantes en el mapa de un país dado

In [7]:
# Trbajamos con un sub dataframe

df_add = df_ta[['restaurant_name', 
               'original_location', 
               'country',
               'region',
               'province',
               'city',
               'address',
               'latitude',
               'longitude']]

df_add

,restaurant_name,original_location,country,region,province,city,address,latitude,longitude
0,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131
1,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480
2,Au Bout du Pont,"[""Europe"", ""France"", ""Centre-Val de Loire"", ""B...",France,Centre-Val de Loire,Berry,Rivarennes,"2 rue des Dames, 36800 Rivarennes France",46.635895,1.386133
3,Le Relais de Naiade,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Co...",France,Nouvelle-Aquitaine,Correze,Lacelle,"9 avenue Porte de la Correze 19170, 19170 Lace...",45.642610,1.824460
4,Relais Du MontSeigne,"[""Europe"", ""France"", ""Occitanie"", ""Aveyron"", ""...",France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,"route du Montseigne, 12620 Saint-Laurent-de-Le...",44.208860,2.960470
...,...,...,...,...,...,...,...,...,...
1083392,Complex Popas Pacurari,"[""Europe"", ""Romania"", ""Northeast Romania"", ""Ia...",Romania,Northeast Romania,Iasi County,None,"Soseaua Pacurari, Valea Lupului 707410 Romania",47.172950,27.519110
1083393,Casa Pastravarului DORIPESCO,"[""Europe"", ""Romania"", ""Transylvania"", ""Central...",Romania,Transylvania,Brasov County,Apata,"DN 13 Judetul Kilometrul 33 Maierus, Apata 507...",45.904423,25.470509
1083394,Hanul Tentea,"[""Europe"", ""Romania"", ""Transylvania"", ""Northwe...",Romania,Transylvania,Maramures County,Sacel,"DN17C, Sacel Romania",47.631920,24.450910
1083395,Casa Paduraru,"[""Europe"", ""Romania"", ""Southern Romania"", ""Arg...",Romania,Southern Romania,Arges County,None,"Sat. Argeselu Numarul 432, Maracineni 117450 R...",44.918950,24.867634


In [42]:
df_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083397 entries, 0 to 1083396
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   restaurant_name    1083397 non-null  object 
 1   original_location  1083397 non-null  object 
 2   country            1083397 non-null  object 
 3   region             1033074 non-null  object 
 4   province           742765 non-null   object 
 5   city               682712 non-null   object 
 6   address            1083397 non-null  object 
 7   latitude           1067607 non-null  float64
 8   longitude          1067607 non-null  float64
dtypes: float64(2), object(7)
memory usage: 74.4+ MB


In [43]:
# Estudio de las direcciones

In [8]:
# Estudiamos los datos que faltan para buscar la manera de calcularlos

# Datos donde la latitud y la longitud son nulas.
# Agregamos también los datos sin ciudad

coor_nuls = df_add['latitude'].isnull()
city_nuls = df_add['city'].isnull()
city_nn = df_add['city'].notnull()
province_nuls = df_add['province'].isnull()
reg_nuls = df_add['region'].isnull()
df_add[coor_nuls & city_nuls]

,restaurant_name,original_location,country,region,province,city,address,latitude,longitude
37444,Le Ku De Ta,"[""Europe"", ""France"", ""Corsica""]",France,Corsica,None,None,"Route de Marina di Fiori, Corsica France",NaN,NaN
37449,a pignatta,"[""Europe"", ""France"", ""Corsica""]",France,Corsica,None,None,"Rue Paoli, Corsica France",NaN,NaN
37453,Brasserie-Pizzeria La Caravelle,"[""Europe"", ""France"", ""Corsica""]",France,Corsica,None,None,"Place du monument, 20200, Corsica France",NaN,NaN
37460,i scalini,"[""Europe"", ""France"", ""Corsica""]",France,Corsica,None,None,"Haut du Village, Corsica France",NaN,NaN
37462,"Piertou, les pirates","[""Europe"", ""France"", ""Corsica""]",France,Corsica,None,None,"Capo Di feno, Corsica France",NaN,NaN
...,...,...,...,...,...,...,...,...,...
1083094,Serenity Beach Bar,"[""Europe"", ""Romania"", ""Southeast Romania"", ""Tu...",Romania,Southeast Romania,Tulcea County,None,"Plaja Sfantu Gheorghe, Sfantu Gheorghe 827195 ...",NaN,NaN
1083216,Italiano Pub,"[""Europe"", ""Romania"", ""Southern Romania"", ""Arg...",Romania,Southern Romania,Arges County,None,Curtea de Arges Romania,NaN,NaN
1083306,Terasa Taraneasca,"[""Europe"", ""Romania"", ""Southeast Romania"", ""Co...",Romania,Southeast Romania,Constanta County,None,"Mamaia, Constanta 900001 Romania",NaN,NaN
1083307,Restaurant Toscana,"[""Europe"", ""Romania"", ""Southeast Romania"", ""Co...",Romania,Southeast Romania,Constanta County,None,"Strada Lotus, Statiunea Mamaia Nord Hotel Oper...",NaN,NaN


#### Análisis coordenadas y ciudades

1. Todos los restaurantes tienen una dirección
2. Un poco más de 15 mil restaurantes no poseen coordenadas. De ellos, solo un poco más de 5 mil no tienen ciudad tampoco

#### El objetivo es determinar las ciudades faltantes (+400 mil) principalmente a partir de las coordenadas

#### Pasos a seguir:

1. Determinar coordenadas a partir de la dirección
2. Si no es satisfactorio, determinar coordenadas a partir de ciudad, provincia o región
3. Determinar las ciudades que faltan a partir de las coordenadas

Utilizaremos la librería GeoPy

In [8]:
# Utilizamos geopy para completar los datos
from geopy.geocoders import Nominatim 
#import time

# Nominatim es el geocoder oficial de Open Street Map 
# Accedemos a su API para poder obtener la información
# Es obligatorio pasar el parámetro user_agent
geo_loc = Nominatim(user_agent="ta_test1", timeout=5) 

# Para evitar error de timeout al solicitar muchas veces la API
#from geopy.extra.rate_limiter import RateLimite
#geocode = RateLimiter(geo_loc.geocode, min_delay_seconds=1)

def normal_add(add):
    #time.sleep(10)
    #print('add: ', add)
    result = geo_loc.geocode(add)
    result = [result.address, result.latitude, result.longitude] if result != None else None
    print('result: ', result)
    return result

In [28]:
%%time 

df_woloc = df_add[coor_nuls & city_nn].sort_values(by='city')
#.groupby('city').value_counts()

df_woloc['add_normal'] = df_woloc.apply(lambda x: normal_add(x['city']), axis=1)

# Guardamos el resultado en un archivo Parquet que podrá ser utilizado luego
df_woloc.to_parquet("coordenadas.parquet", compression='gzip')

#df_woloc['city'].value_counts()

#df_woloc.to_csv("coordenadas.csv")

#lugar='calle 2 la grita'
#print(normal_add(lugar))

  
#locname = geoLoc.reverse("26.7674446, 81.109758") 
  
#print(locname.address) 

result:  ["'s-Gravenzande, Westland, Zuid-Holland, Nederland", 52.0028294, 4.1603759]
result:  ['A dos Cunhados, A dos Cunhados e Maceira, Torres Vedras, Lisboa, 2560-003 A DOS CUNHADOS, Portugal', 39.1519962, -9.2979195]
result:  ['Aabenraa, Aabenraa Kommune, 6200, Danmark', 55.0446228, 9.4209667]
result:  ['Aachen, Städteregion Aachen, Nordrhein-Westfalen, Deutschland', 50.776351, 6.083862]
result:  ['Aachen, Städteregion Aachen, Nordrhein-Westfalen, Deutschland', 50.776351, 6.083862]
result:  ['Aalen, Verwaltungsgemeinschaft Aalen, Ostalbkreis, Baden-Württemberg, Deutschland', 48.8362705, 10.0931765]
result:  ['Aalen, Verwaltungsgemeinschaft Aalen, Ostalbkreis, Baden-Württemberg, Deutschland', 48.8362705, 10.0931765]
result:  ['Aalsmeer, Noord-Holland, Nederland', 52.2658344, 4.766170039669312]
result:  ['Aalsmeer, Noord-Holland, Nederland', 52.2658344, 4.766170039669312]
result:  ['Aarhus, Aarhus Kommune, 8000, Danmark', 56.1496278, 10.2134046]
result:  ['Aarhus, Aarhus Kommune, 80

In [6]:
%%time 
df_woloc = pd.read_parquet('coordenadas.parquet', engine='fastparquet')

CPU times: user 95.9 ms, sys: 8.04 ms, total: 104 ms
Wall time: 113 ms


In [7]:
df_woloc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10637 entries, 158044 to 250850
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_name    10637 non-null  object 
 1   original_location  10637 non-null  object 
 2   country            10637 non-null  object 
 3   region             9874 non-null   object 
 4   province           7819 non-null   object 
 5   city               10637 non-null  object 
 6   address            10637 non-null  object 
 7   latitude           0 non-null      float64
 8   longitude          0 non-null      float64
 9   add_normal         10487 non-null  object 
dtypes: float64(2), object(8)
memory usage: 914.1+ KB


In [9]:
add_nor_nul = df_woloc['add_normal'].isnull()

df_woloc[add_nor_nul]

df_woloc.loc[add_nor_nul,'add_normal'] = df_woloc[add_nor_nul].apply(lambda x: normal_add(x['address']), axis=1)

result:  ['Ελλάς', 38.9953683, 21.9877132]
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  ['Chemin du Vert Galant, Puy Fournier, Brantôme, Brantôme en Périgord, Nontron, Dordogne, Nouvelle-Aquitaine, France métropolitaine, 24310, France', 45.3607794, 0.6483832]
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result:  None
result

In [44]:
df_woloc[df_woloc['add_normal'].isnull()]

df_woloc.loc[add_nor_nul,'add_normal'] = df_woloc[add_nor_nul].apply(lambda x: normal_add(x['province']), axis=1)

df_woloc.to_csv("coordenadas.csv")

result:  ['Telos (Dodecanese), Μεγάλο Χωριό, Δήμος Τήλου, Περιφερειακή Ενότητα Ρόδου, Περιφέρεια Νοτίου Αιγαίου, Αποκεντρωμένη Διοίκηση Αιγαίου, 85002, Ελλάς', 36.4570884, 27.34309585792063]
result:  ['Περιφερειακή Ενότητα Λακωνίας, Περιφέρεια Πελοποννήσου, Αποκεντρωμένη Διοίκηση Πελοποννήσου, Δυτικής Ελλάδας και Ιονίου, Ελλάς', 36.76027945, 22.79046431389755]
result:  ['Κέρκυρα, Περιφερειακή Ενότητα Κέρκυρας, Περιφέρεια Ιονίων Νήσων, Αποκεντρωμένη Διοίκηση Πελοποννήσου, Δυτικής Ελλάδας και Ιονίου, Ελλάς', 39.591337, 19.859618918733858]
result:  ['Κέρκυρα, Περιφερειακή Ενότητα Κέρκυρας, Περιφέρεια Ιονίων Νήσων, Αποκεντρωμένη Διοίκηση Πελοποννήσου, Δυτικής Ελλάδας και Ιονίου, Ελλάς', 39.591337, 19.859618918733858]
result:  ['Κέρκυρα, Περιφερειακή Ενότητα Κέρκυρας, Περιφέρεια Ιονίων Νήσων, Αποκεντρωμένη Διοίκηση Πελοποννήσου, Δυτικής Ελλάδας και Ιονίου, Ελλάς', 39.591337, 19.859618918733858]
result:  ['Σέρρες, Δήμος Σερρών, Περιφερειακή Ενότητα Σερρών, Περιφέρεια Κεντρικής Μακεδονίας, Απ

In [10]:
df_woloc[df_woloc['add_normal'].isnull()]

,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,add_normal
index,,,,,,,,,,
959715,Mastoras,"[""Europe"", ""Greece"", ""Peloponnese"", ""Laconia R...",Greece,Peloponnese,Laconia Region,Aggelona,"Aggelona, Monemvasia Municipality 230 70 Greece",NaN,NaN,None
986076,Marina,"[""Europe"", ""Greece"", ""Ionian Islands"", ""Corfu""...",Greece,Ionian Islands,Corfu,Agios Georgios Pagon,Agios Georgios Pagon 490 83 Greece,NaN,NaN,None
975592,Bougiabessa,"[""Europe"", ""Greece"", ""Ionian Islands"", ""Corfu""...",Greece,Ionian Islands,Corfu,Agios Ioannis Peristeron,Agios Ioannis Peristeron Greece,NaN,NaN,None
975591,Blue N’ Bar,"[""Europe"", ""Greece"", ""Ionian Islands"", ""Corfu""...",Greece,Ionian Islands,Corfu,Agios Ioannis Peristeron,Agios Ioannis Peristeron 490 84 Greece,NaN,NaN,None
973654,Chasapotaverna Kyra Maria,"[""Europe"", ""Greece"", ""Central Macedonia"", ""Ser...",Greece,Central Macedonia,Serres Region,Agkistro,Agkistro Greece,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...
301811,Ischler Alm,"[""Europe"", ""Austria"", ""Lower Austria"", ""Voesen...",Austria,Lower Austria,None,Voesendorf,"SCS Eingang 3, Top 234, Voesendorf 2334 Austria",NaN,NaN,None
301823,"Asia Restaurant WOK Town, Hu & Wang Ges.mb.H","[""Europe"", ""Austria"", ""Lower Austria"", ""Voesen...",Austria,Lower Austria,None,Voesendorf,"Shopping City Süd Galerie 225, Shop 281, Voese...",NaN,NaN,None
315576,Piestro Genussbar,"[""Europe"", ""Austria"", ""Lower Austria"", ""Woelle...",Austria,Lower Austria,None,Woellersdorf-Steinabrueckl,"Marktplatz 6, Woellersdorf-Steinabrueckl Austria",NaN,NaN,None


In [11]:
df_woloc.to_parquet("coordenadas.parquet", compression='gzip')

### De aquí para abajo son pruebas

In [ ]:
%%time

# Nunca lo terminé de ejecutar, pero es infinitamente más lento que
# tratar columna por columna
        
for i in df_aw.index: 
    print('i: ', i)
    df_aw.loc[i,awards_list] = df_aw.apply(lambda col: aw_by_rest(str(df_aw.loc[i]['awards']), str(col.name)), axis=0) 
    
df_aw.to_csv("lineas.csv")
#df_aw

In [ ]:
%%time
import numpy as np
awards = df_ta.awards.unique().tolist()
aw_list = []

for aw in awards:
    aw_list = aw_list + str(aw).split(',')
    
s_awards = pd.Series(aw_list)
s_awards

In [ ]:
# Después de una primera lectura, vamos a generar un archivo de texto con las
# columnas y sus tipos para luego indicar un tipo preciso al cargar el archivo

def col_type_csv(df, file_name):
    result = pd.Series(df.dtypes)
    result.to_csv(file_name + '.csv')
    
col_type_csv(df_ta, 'columns_types')

In [ ]:
# Generamos un diccionario con los nuevos tipos de datos
dict_types = pd.read_csv('columns_types1.csv', index_col=0, squeeze=True).to_dict()

print(dict_types)

In [ ]:
# Trabajamos en el archivo csv cada tipo de dato, usamos la info de describe()
# Agregamos los tipos de columna al momento del load

df_types = pd.read_csv('columns_types1.csv', index_col=0, names=('type',))

col_to_chg = ['open_days_per_week', 'total_reviews_count', 'reviews_count_in_default_language', 
              'excellent', 'very_good', 'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere']

for col in col_to_chg:
    df_ta[col] = df_ta[col].fillna(0)

for col in df_ta.columns:
    df_ta[col] = df_ta[col].astype(df_types.loc[col].type)

df_ta.dtypes